# Парсер

#### Импортируем и настраиваем все

In [2]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import chromedriver_binary 
import urllib
import time
from selenium.webdriver.common.keys import Keys
import lxml
import requests
import base64
import json
import pandas as pd
import numpy as np
import urllib.request
from fake_useragent import UserAgent
from bs4 import BeautifulSoup 
from selenium.webdriver.common.action_chains  import ActionChains

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

ModuleNotFoundError: No module named 'chromedriver_binary'

#####  Подготовка к работе с API, авторизируемся на сайте и изменяем кодировку для получения токена доступа.

In [105]:
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}

message = f"{'3e661d7345d04909b518ec04ec996fd2'}:{'c3ce130028654a779d6404a619845bd6'}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']

In [106]:
token

'BQAsoyv_4LYEJCJ5Z5zI3S_UN0akV_zVojqgrBKv85MFiucmqajOXguSDwrTnRkwKhHx5GYd7_fC5LyDFtQ'

Успешно получен токен.

## Пишем функции для получения и обработки данных

###### Функция получения сведений о треках, находящихся в одном плейлисте (и ее мини-версия)

In [107]:
def get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = playlist_req_to_list (page)    
    return page


In [108]:
def mini_get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = mini_playlist_req_to_list(page)    
    return page

###### Функция получения параметров песни

In [109]:
def song_paramets(song_id):
    
    song_url = f'https://api.spotify.com/v1/audio-features/{song_id}'
    headers = {
        "Authorization": "Bearer " + token
    }

    page = requests.get(url=song_url, headers=headers)
    page =  page.json()
    return page

###### Функция для распредения инфомации о песнях из json-формата в DataFrame-таблицу (и ее мини-версия)

In [110]:
def playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
    duration = [None for i in range(50)]
    artist_name = [None for i in range(50)]
    artist_id = [None for i in range(50)]
    album_name = [None for i in range(50)]
    album_id = [None for i in range(50)]
    release_date = [None for i in range(50)]
    local_position = [None for i in range(50)]
    world_position = [None for i in range(50)]
    danceability = [None for i in range(50)]
    energy = [None for i in range(50)]
    key = [None for i in range(50)]
    loudness = [None for i in range(50)]
    mode = [None for i in range(50)]
    speechiness = [None for i in range(50)]
    acousticness = [None for i in range(50)]
    instrumentalness = [None for i in range(50)]
    liveness = [None for i in range(50)]
    valence = [None for i in range(50)]
    tempo = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        duration[i] = item['track']['duration_ms']
        local_position[i] = item['track']['popularity']       
        artist_name[i] = item['track']['album']['artists'][0]['name']
        artist_id[i] = item['track']['album']['artists'][0]['id']
        album_name[i] = item['track']['album']['name']
        album_id[i] = item['track']['album']['id']
        release_date[i] = item['track']['album']['release_date']
        
        parametrs = song_paramets(song_id[i])
        
        danceability[i] = parametrs['danceability']
        energy[i] = parametrs['energy']
        key[i] = parametrs['key']
        loudness[i] = parametrs['loudness']
        mode[i] = parametrs['mode']
        speechiness[i] = parametrs['speechiness']
        acousticness[i] = parametrs['acousticness']
        instrumentalness[i] = parametrs['instrumentalness']
        liveness[i] = parametrs['liveness']
        valence[i] = parametrs['valence']
        tempo[i] = parametrs['tempo']
        
        i += 1
        
    data_dict = {'song_name' : song_name,
                 'song_id' : song_id,
                 'duration_ms' : duration,
                 'local_position' : local_position,
                 'artist_name' : artist_name,
                 'artist_id' : artist_id,
                 'album_name' : album_name,
                 'album_id' : album_id,
                 'release_date' : release_date,
                 'danceability': danceability,
                 'energy': energy,
                 'key': key,
                 'loudness': loudness,
                 'mode': mode,
                 'speechiness': speechiness,
                 'acousticness': acousticness,
                 'instrumentalness': instrumentalness,
                 'liveness': liveness,
                 'valence': valence,
                 'tempo': tempo
                }
    data_set = pd.DataFrame(data_dict)
    
    return data_set

In [111]:
def mini_playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        
        i += 1
        
    data_dict = {'song_name' : song_name, 'song_id' : song_id}
    data_set = pd.DataFrame(data_dict)
    
    return data_set

##### Функция по получению основной инфы о плейлисте

In [112]:
def playlist_info(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    text = page['description']
    number = page['followers']['total']
    country = text[text.find('now -')+6:-1]
    info = [playlist_id,country,number]
    return info

##### Функция по получению для каждого плейлиста списка песен, их места в топе, прослушиваний и т.д.

In [113]:
def get_MainInfo(playlist_id):
    #создаем часть таблицы (место в топе в качестве индекса, имена и id песен)
    MainInfo = mini_get_playlist(playlist_id)[['song_name', 'song_id']]
    
    #парсим код страницы альбома с испльзованием селениума
    site = "https://open.spotify.com/playlist/" + playlist_id
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(site)
    time.sleep(3)
    response = driver.page_source
    driver.close()
    tree = BeautifulSoup(response, 'html.parser')
    
    #ищем все куски кода, где написаны просмотры
    a = tree.find_all('div', class_ = 'd47b790d001ed769adcd9ddfc0e83acc-scss f3fc214b257ae2f1d43d4c594a94497f-scss')
    
    #из них получаем строки с прослушиваниями и обрабатываем их
    res = []
    b = list(a)
    for i in b:
        views = str(i)[-16:-6].replace('\xa0', '')
        res.append(views)
    #res - список прослушиваний от топа ко дну (неформатированный)
    #вытащим из элементов res только числа
    views = []
    for i in res:
        ddd = [s for s in i.split('>') if s.isdigit()]
        views.append(int(ddd[0]))
    #views - просмотры по порядку для каждой песни из топа
    #добавляем колонки с просмотрами, местом в топе и страной к существующей таблице
    MainInfo.insert(2, "views", views)
    MainInfo.insert(0, "place", range(1,51))
    MainInfo.insert(0, "country", playlist_info(playlist_id)[1])
    return MainInfo

##### функция для получения жанров по исполнителю

In [114]:
def get_genres(artists):
    result = []
    for i in tqdm(artists):
        id = i
        playlistUrl = f'https://api.spotify.com/v1/artists/{id}'
        headers = {"Authorization": "Bearer " + token}
        page = requests.get(url=playlistUrl, headers=headers)
        page = page.json()
        genres = page['genres']
        result.append(genres)
        time.sleep(0.2)
    return result

## Находим все айдишники альбомов 
#### Полученное ids-список айдишников

In [115]:
site = "https://open.spotify.com/genre/charts-regional-weekly"

#chrome_options = Options() работает у Дениса
#chrome_options.add_argument("--headless")   настройка на всякий случай

#настраиваем селениум
driver = webdriver.Chrome('D:\chromedriver.exe') #работает у Дениса
#driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver') #работает у Сани
#открываем сайт, двигаем мышку и кликаем, чтобы потом заработала прокрутка
driver.get(site)
driver.maximize_window()
time.sleep(3)
ActionChains(driver).move_by_offset(1520, 350).click().perform() #у дениса 1520 350
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) 
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) #моя добавка
time.sleep(3)


#получаем код страницы и делаем суп
response = driver.page_source
driver.close()
tree = BeautifulSoup(response, 'html.parser')

In [116]:
#ищем все ссылки на странице
a = tree.find_all('a', class_ = 'f7ebc3d96230ee12a84a9b0b4b81bb8f-scss')

#из блоков с ссылками достаем айдишники плейлистов
ids = []
for i in a:
    id = i.get('href')
    ids.append(id[10:])

# Cобираем датасеты

### Структура данных:
##### 1) Маленький датасет вида (айди листа, страна, кол-во подписчиков)
##### 2) Датасет побольше (данные о каждой песне, включая ее акустические характеристики)
##### 3) Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

### Маленький датасетик


In [118]:
countries = pd.DataFrame(playlist_info(item) for item in ids)
countries.columns= ['id', 'country', 'followers'] 
countries = countries.set_index('country')


### Датасет побольше (данные о каждой песне)


In [119]:
allsongs=pd.DataFrame()
for item in tqdm(ids):
    allsongs=pd.concat([allsongs,get_playlist(item)],sort=False, axis=0, )

allsongs = allsongs.drop_duplicates(subset=['song_id'])
allsongs = allsongs.set_index('song_id')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:22<00:00, 16.49s/it]


In [120]:
#добавим инфу о жанрах в allsongs
artists = list(allsongs['artist_id'])
genres = get_genres(artists)
allsongs['genres'] = genres

100%|████████████████████████████████████████████████████████████████████████████████| 147/147 [02:18<00:00,  1.06it/s]


### Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

In [121]:
positions = pd.DataFrame()
for id in tqdm(ids):
    country = get_MainInfo(id)
    positions = pd.concat([positions, country],sort=False, axis=0)
    
positions = positions.reset_index(drop = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:08<00:00, 25.62s/it]


### День третий

In [62]:
import csv
import pandas as pd

df_pos = pd.read_csv('positions 2.csv',  sep=',', decimal='\t')
df_songs = pd.read_csv('allsongs.csv',  sep=',', decimal='\t')


### Добавляем дополнительную информацию о треках в таблицу с чартами

In [63]:
df = pd.merge(df_pos, df_songs, left_on = 'song_id', right_on = 'song_id', how = 'left')

#### Чуть-чуть изменим названия колонок

In [64]:
df.columns

Index(['Unnamed: 0', 'country', 'place', 'song_name_x', 'song_id', 'views',
       'song_name_y', 'duration_ms', 'local_position', 'artist_name',
       'artist_id', 'album_name', 'album_id', 'release_date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'genres'],
      dtype='object')

In [65]:
df = df.rename(columns = {"place": "position",
                                 "country" : "region",
                                 "local_position" : "popularity",
                                 "song_name_x" : "song_name"
                         })
df = df.drop(['song_name_y','Unnamed: 0'], axis = 1)
df

,region,position,song_name,song_id,views,duration_ms,popularity,artist_name,artist_id,album_name,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genres
0,Global,1,good 4 u,4ZtFanR9U6ndgddUvNcjcG,84131760,178146,93,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,...,9,-5.044,1,0.154,0.335,0.0,0.0849,0.688,166.928,['pop']
1,Global,2,Butter,3VqeTFIvhxu3DIe4eZVzGq,58122349,164441,98,BTS,3Nrfpe0tUJi4K4DXYWgMUX,Butter,...,8,-5.187,1,0.0948,0.00323,0.0,0.0906,0.695,109.997,"['k-pop', 'k-pop boy group']"
2,Global,3,deja vu,6HU7h9RYOaPRFeh0R3UeAr,45060633,215506,89,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,SOUR,...,2,-7.222,1,0.112,0.584,5.7e-06,0.37,0.178,180.917,['pop']
3,Global,4,MONTERO (Call Me By Your Name),67BtfxlNbhBmCDR2L2l8qd,38927630,137875,100,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,MONTERO (Call Me By Your Name),...,8,-6.682,0,0.152,0.297,0.0,0.384,0.758,178.818,"['lgbtq+ hip hop', 'pop rap']"
4,Global,5,Kiss Me More (feat. SZA),748mdHapucXQri7IAO8yFK,36768939,208866,98,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Kiss Me More (feat. SZA),...,8,-3.541,1,0.0286,0.235,0.000158,0.123,0.742,110.968,"['dance pop', 'pop', 'pop rap']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,Japan,46,snow jam,6uUtmCz8XXep7n1QmC1jJG,733439,185378,71,Rin音,2sd5k8N8cAOm6Q8OCcePw4,swipe sheep,...,1,-5.842,1,0.201,0.185,0.0,0.159,0.805,172.016,"['j-pop', 'japanese pop rap', 'japanese teen p..."
2646,Japan,47,恋人ごっこ,2SihT7AVMiI2VtaAyVIcnO,729847,200219,71,Macaroni Empitsu,1t17z3vfuc82cxSDMrvryJ,hope,...,1,-5.301,1,0.0361,0.124,0.0,0.0603,0.745,102.033,"['anime', 'j-pop', 'j-rock', 'japanese alterna..."
2647,Japan,48,点描の唄,1sIIlVrnPhrvmTrHtzM7tV,726701,307066,71,Mrs. GREEN APPLE,4QvgGvpgzgyUOo8Yp8LDm9,青と夏,...,0,-4.612,1,0.0273,0.447,0.0,0.103,0.308,152.014,"['anime rock', 'j-pop', 'j-rock']"
2648,Japan,49,マリーゴールド,2cKiHG16JBiqHdi7Llq0r7,704439,306626,70,Aimyon,5kVZa4lFUmAQlBogl1fkd6,瞬間的シックスセンス,...,2,-3.098,1,0.0286,0.0452,0.0,0.0907,0.413,106.094,"['j-acoustic', 'j-pop']"


#### Преобразуем столбец "genres" из строки в лист

In [66]:
import copy

def str_to_list(string):
    s = list(map(lambda w : w.strip(" ").strip("'"), string.strip('[').strip(']').split(',')))
    return(s)
df_1 = copy.deepcopy(df)
df_1['genres'] = df['genres'].apply(str_to_list)

### Кажется, сейчас самое время поискать пропущенные значения

In [67]:
df_1.isnull().values.any()

False

#### Пробегаемся глазами по таблице и видим, что в ней есть нули в колонках "key", "mode" и "instrumentalness", а так же пустые листы в столбце "genres", но:

- "key" - булевая переменная, все ок;
- "mode" - тональность, ноль - тоже значение, все ок;
- "instrumentalness" - значение, которое параметр может принимать, все ок;
- "genres" - жанры отстутсвуют, не ок, попробуем найти записи и посмотреть, что за песни.

In [68]:
from collections import Counter

genres = []
for i in range(2650):
    genres.extend(df_1['genres'][i])
    
cnt = Counter(genres)

sorted_dict = {}
sorted_keys = sorted(cnt, key=cnt.get, reverse = True)  # [1, 3, 2]

for w in sorted_keys:
    sorted_dict[w] = cnt[w]
    
sorted_dict


{'pop': 719,
 'trap latino': 282,
 'dance pop': 279,
 'reggaeton': 238,
 'latin': 222,
 'k-pop': 142,
 'pop rap': 139,
 '': 134,
 'canadian pop': 127,
 'k-pop boy group': 119,
 'pop dance': 115,
 'post-teen pop': 111,
 'tropical house': 104,
 'reggaeton colombiano': 91,
 'house': 79,
 'edm': 79,
 'uk pop': 76,
 'rap': 73,
 'italian pop': 63,
 'canadian contemporary r&b': 62,
 'russian hip hop': 62,
 'uk dance': 58,
 'colombian pop': 56,
 'lgbtq+ hip hop': 55,
 'australian hip hop': 48,
 'german hip hop': 48,
 'russian trap': 47,
 'indie rock italiano': 45,
 'eurovision': 45,
 'j-pop': 44,
 'puerto rican pop': 40,
 'north carolina hip hop': 39,
 'conscious hip hop': 37,
 'trance': 36,
 'reggaeton flow': 35,
 'mandopop': 34,
 'rap maroc': 34,
 'slap house': 31,
 'electropop': 31,
 'norwegian pop': 30,
 'cantopop': 29,
 'uk hip hop': 28,
 'dutch pop': 28,
 'latin hip hop': 28,
 'v-pop': 28,
 'czsk hip hop': 28,
 'london rap': 25,
 'panamanian pop': 24,
 'egyptian hip hop': 24,
 'polish hi

#### Видим, что Поп-музыка с большим отрывом выигрывает у остальных жанров, поэтому все пропущенные записи заполняем строкой ['pop'] (позже мы все строки преобразуем в листы)

In [69]:
df[df['genres'] == '[]'] = "['pop']"

In [70]:
def str_to_list(string):
    s = list(map(lambda w : w.strip(" ").strip("'"), string.strip('[').strip(']').split(',')))
    return(s)

df['genres'] = df['genres'].apply(str_to_list)

#### Ура мы собрали датасет, в котором пропущенные значения отсутствуют, сохраним его

In [76]:
df.to_csv('complete.csv', index = False)

### Расчитываем производные показатели

- 
-
-
-
-
